# IEFI Lautaro Santos Da Silveira

En el siguiente trabajo, se propone el desafío de identificar si una noticia es o no una Fake News. 
Para tomar esa decisión primero se deberá aplicar herramientas de Procesamiento del Lenguaje Natural (NLP), y luego se procederá a a la eleccion del modelo que mejor aprenda. Se contará con modelos de SVM, RandomForest, Arbol de desicion y Regresion Logistica.

### Librerias

In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
from nltk import regexp_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
!python -m spacy download es_core_news_sm
import spacy
nlp= spacy.load('es_core_news_sm')
import re
from collections  import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


     ---------------------------------------- 12.9/12.9 MB 4.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


### Set de datos

In [2]:
#df= pd.read_excel(r'D:\Escritorio\IES21\4to cuatrimestre\PLN\Noticias Verdaderas y Falsas.xlsx')
df= pd.read_excel(r'C:\Users\Usuario\Desktop\lautaro\ies 21\IES21\4to cuatrimestre\PLN\Noticias Verdaderas y Falsas.xlsx')
df

,class,Text
0,True,Algunas de las voces extremistas más conocida...
1,True,Después de casi dos años y medio de luchas po...
2,True,Dos periodistas birmanos de la agencia Reuters...
3,True,El Cuerpo Nacional de Policía ha detenido a cu...
4,True,El desfile de la firma en Roma se convierte en...
...,...,...
1995,True,El Consejo de Gobierno ha dado su visto bueno...
1996,True,Investigadores valencianos han desarrollado u...
1997,True,Los arrestados actuaban en coches y en establ...
1998,True,El Rey ha encargado este miércoles a Pedro Sá...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   class   2000 non-null   bool  
 1   Text    2000 non-null   object
dtypes: bool(1), object(1)
memory usage: 17.7+ KB


### Funcion de preprocesamiento

In [4]:
stops= set(stopwords.words('spanish'))
def preprocesamiento(documento):
# Eliminación de valores numéricos
  documento = re.sub('\d', ' ', documento)
  documento = documento.lower()
############################## TOKENIZAR ##############################
# Tokenizo con una List Comprehension, y luego a esta le filtro las stopwords
  tokenizado = [token for token in regexp_tokenize(documento, pattern='\w+')]
  filtro = [filtro for filtro in tokenizado if filtro not in stops]
############################## LEMATIZAR ##############################
# Spacy tiene su propio tokenizador 
  doc= nlp(documento)
# Lematizo al texto, para eliminár los signos de puntuación o demas signos que no me aportarán al análisis, se filtrarán los tokens
# que están en la anterior List Comprehension llamada filtro
  lemma = [lema.lemma_ for lema in doc if str(lema) in filtro]
  Procesada=" ".join(map(str, lemma))
  return Procesada

In [5]:
pre_procesamiento=df['Text'].apply(preprocesamiento)
pre_procesamiento

0       voz extremista conocido cuenta asociado movimi...
1       después casi dos año medio lucha política judi...
2       dos periodista birmano agencia reuters liberar...
3       cuerpo nacional policía detener cuatro persona...
4       desfile firma roma convertir oda libertad muje...
                              ...                        
1995    consejo gobierno dar visto bueno convocatoria ...
1996    investigador valenciano desarrollar innovador ...
1997    arrestado actuar coche establecimiento policía...
1998    rey encargar miércoles pedro sánchez presentar...
1999    prueba realizado centro nacional microbiología...
Name: Text, Length: 2000, dtype: object

#### palabras mas frecuentes

In [6]:
'''
def ExtraeFeatures(documentos):
    features_aux = []
    features = []
    for documento in documentos:
        for feature in documento:
            features_aux.append(feature)
    for key in Counter(features_aux):
        features.append(key)
    return features

palabras = ExtraeFeatures(pre_procesamiento)
len(palabras)
'''

'\ndef ExtraeFeatures(documentos):\n    features_aux = []\n    features = []\n    for documento in documentos:\n        for feature in documento:\n            features_aux.append(feature)\n    for key in Counter(features_aux):\n        features.append(key)\n    return features\n\npalabras = ExtraeFeatures(pre_procesamiento)\nlen(palabras)\n'

In [7]:
'''
from itertools import chain
lista= list(chain( * pre_procesamiento ))
x=Counter(lista).most_common()
dict_1=dict()
for key,values in x: 
    dict_1.setdefault(key, values)
dict_1 
df1 = pd.DataFrame([[key, dict_1[key]] for key in dict_1.keys()], columns=['Name', 'Cant'])
grupo=df.groupby(['Cant'])['Cant'].count().reset_index(name='cantidad')
len(lista)
'''

"\nfrom itertools import chain\nlista= list(chain( * pre_procesamiento ))\nx=Counter(lista).most_common()\ndict_1=dict()\nfor key,values in x: \n    dict_1.setdefault(key, values)\ndict_1 \ndf1 = pd.DataFrame([[key, dict_1[key]] for key in dict_1.keys()], columns=['Name', 'Cant'])\ngrupo=df.groupby(['Cant'])['Cant'].count().reset_index(name='cantidad')\nlen(lista)\n"

#### continua

In [8]:
vectorizer = TfidfVectorizer()
vectores = vectorizer.fit_transform(pre_procesamiento).toarray()
nombres= vectorizer.get_feature_names_out()
df_vector= pd.DataFrame(vectores,columns=nombres )
df_vector


,abajo,abalanzar,abanderar,abandonado,abandonar,abandono,abascal,abasccal,abateír,abatir,...,óscar,últi,últimamente,último,único,útil,рог,сото,тара,те
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
df_vector['class']=df['class']
df_vector

,abajo,abalanzar,abanderar,abandonado,abandonar,abandono,abascal,abasccal,abateír,abatir,...,últi,últimamente,último,único,útil,рог,сото,тара,те,class
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
1996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
1997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
1998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True


In [10]:
df_vector=shuffle(df_vector, random_state=123)

In [11]:
x= df_vector.drop('class',axis=1)
y= df_vector['class']

In [12]:
x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.2,random_state=123)

### Modelos

In [13]:
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, cross_validate
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [14]:
# Creamos los folds externos
skfold_outer = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)

# Creamos los folds internos
skfold_inner = StratifiedKFold(n_splits=3, shuffle=True, random_state=123)

#### Arbol de Decision

In [15]:
# Instanciamos el estimador
arbol= DecisionTreeClassifier(random_state=123)

# Creamos la grilla de hiperparametros a probar
param_grid_arbol = [{'max_depth':[35,40,45], 'criterion':['gini','entropy']}]

# Creamos el inner CV (GSCV para buscar los mejores hiperparámetros con los k folds internos)
gscv_arbol = GridSearchCV(estimator=arbol,
                    param_grid=param_grid_arbol, 
                    scoring='accuracy',
                    n_jobs=-1,
                    cv=skfold_inner,
                    verbose=0,
                    refit=True)

# Ahora creamos el outer CV para evaluar el inner CV para cada fold externo
# Recordar que cross_val_score/cross_validate realiza el .fit y .predict internamente
# Tener en cuenta que el .predict lo realiza sobre el modelo que tuvo mejores resultados en el entrenamiento (gscv_knn.best_estimator_)
nested_cv_arbol = cross_validate(gscv_arbol, X=x, y=y, cv=skfold_outer, return_train_score=True)

result_cv_arbol = pd.DataFrame(nested_cv_arbol)
result_cv_arbol

,fit_time,score_time,test_score,train_score
0,345.853230,0.264001,0.6425,0.866875
1,470.574492,0.160007,0.6475,0.849375
2,303.671870,0.229440,0.6575,0.800625
3,339.033980,0.135881,0.6125,0.818125
4,322.714527,0.197015,0.6125,0.804375


#### SVM

In [19]:
# Instanciamos el estimador
svc= SVC(random_state=123)

# Creamos la grilla de hiperparametros a probar
param_grid_svc = [{'C':[0.0001,0.01,0.1,1], 'gamma':['scale','auto']}]

# Creamos el inner CV (GSCV para buscar los mejores hiperparámetros con los k folds internos)
gscv_svc = GridSearchCV(estimator=svc,
                    param_grid=param_grid_svc, 
                    scoring='accuracy',
                    n_jobs=-1,
                    cv=skfold_inner,
                    verbose=0,
                    refit=True)

# Ahora creamos el outer CV para evaluar el inner CV para cada fold externo
# Recordar que cross_val_score/cross_validate realiza el .fit y .predict internamente
# Tener en cuenta que el .predict lo realiza sobre el modelo que tuvo mejores resultados en el entrenamiento (gscv_knn.best_estimator_)
nested_cv_svc = cross_validate(gscv_svc, X=x, y=y, cv=skfold_outer, return_train_score=True)

result_cv_svc = pd.DataFrame(nested_cv_svc)
result_cv_svc

,fit_time,score_time,test_score,train_score
0,293.568247,7.035432,0.7600,0.999375
1,299.334985,7.187690,0.7825,1.000000
2,311.202961,8.936035,0.7575,0.999375
3,270.767579,8.928568,0.8150,1.000000
4,269.762343,7.573648,0.8275,1.000000


#### Random Forest

In [22]:
# Instanciamos el estimador
random= RandomForestClassifier(random_state=123)

# Creamos la grilla de hiperparametros a probar
param_grid_random = [{'n_estimators':[35,40,45], 'criterion':['gini','entropy'], 'max_depth':[35,40,45]}]

# Creamos el inner CV (GSCV para buscar los mejores hiperparámetros con los k folds internos)
gscv_random = GridSearchCV(estimator=random,
                    param_grid=param_grid_random, 
                    scoring='accuracy',
                    n_jobs=-1,
                    cv=skfold_inner,
                    verbose=0,
                    refit=True)

# Ahora creamos el outer CV para evaluar el inner CV para cada fold externo
# Recordar que cross_val_score/cross_validate realiza el .fit y .predict internamente
# Tener en cuenta que el .predict lo realiza sobre el modelo que tuvo mejores resultados en el entrenamiento (gscv_knn.best_estimator_)
nested_cv_random = cross_validate(gscv_random, X=x, y=y, cv=skfold_outer, return_train_score=True)

result_cv_random = pd.DataFrame(nested_cv_random)
result_cv_random

,fit_time,score_time,test_score,train_score
0,301.761676,0.182981,0.6600,0.966875
1,279.123584,0.137009,0.7100,0.964375
2,263.611159,0.124996,0.6600,0.972500
3,261.795267,0.099033,0.7200,0.953125
4,264.132137,0.102997,0.7025,0.966875


#### Regresion Logistica

In [25]:
# Instanciamos el estimador
RL= LogisticRegression(random_state=123)

# Creamos la grilla de hiperparametros a probar
param_grid_RL = [{'C':[0.0001,0.01,0.1,1], 'penalty':['l1','l2','elasticnet','none']}]

# Creamos el inner CV (GSCV para buscar los mejores hiperparámetros con los k folds internos)
gscv_RL = GridSearchCV(estimator=RL,
                    param_grid=param_grid_RL, 
                    scoring='accuracy',
                    n_jobs=-1,
                    cv=skfold_inner,
                    verbose=0,
                    refit=True)

# Ahora creamos el outer CV para evaluar el inner CV para cada fold externo
# Recordar que cross_val_score/cross_validate realiza el .fit y .predict internamente
# Tener en cuenta que el .predict lo realiza sobre el modelo que tuvo mejores resultados en el entrenamiento (gscv_knn.best_estimator_)
nested_cv_RL = cross_validate(gscv_RL, X=x, y=y, cv=skfold_outer, return_train_score=True)

result_cv_RL = pd.DataFrame(nested_cv_RL)
result_cv_RL

c:\Users\Usuario\Desktop\lautaro\ies 21\entorno\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
24 fits failed out of a total of 48.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
12 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Usuario\Desktop\lautaro\ies 21\entorno\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Usuario\Desktop\lautaro\ies 21\entorno\lib\site-packages\sklearn\linear_model\_logistic.py", line 1094, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "c:\Users\Usuario\Desktop\lautaro\ies 21\entorno\lib\site-packag

,fit_time,score_time,test_score,train_score
0,26.165942,0.184003,0.7350,0.971250
1,16.744107,0.091015,0.7700,0.975000
2,16.015795,0.104002,0.7500,0.976875
3,14.159440,0.089303,0.7800,0.974375
4,14.456590,0.120014,0.8075,0.975625


#### Comparacion de modelos

In [28]:
Arbol_promedio = result_cv_arbol.test_score.mean()
Arbol_desvio = result_cv_arbol.test_score.std()

SVC_promedio = result_cv_svc.test_score.mean()
SVC_desvio = result_cv_svc.test_score.std()

RandomForest_promedio = result_cv_random.test_score.mean()
RandomForest_desvio = result_cv_random.test_score.std()

RL_promedio = result_cv_RL.test_score.mean()
RL_desvio = result_cv_RL.test_score.std()

In [31]:
print('Arbol promedio en el outer: ',Arbol_promedio, " +/- ", 2*Arbol_desvio, " (95%)")
print('SVC promedio en el outer: ',SVC_promedio, " +/- ", 2*SVC_desvio, " (95%)")
print('Random Forest promedio en el outer: ',RandomForest_promedio, " +/- ", 2*RandomForest_desvio, " (95%)")
print('Regresion Logistica promedio en el outer: ',RL_promedio, " +/- ", 2*RL_desvio, " (95%)")

Arbol promedio en el outer:  0.6345000000000001  +/-  0.04159326868617076  (95%)
SVC promedio en el outer:  0.7885  +/-  0.0635019684734261  (95%)
Random Forest promedio en el outer:  0.6905  +/-  0.05705260730238359  (95%)
Regresion Logistica promedio en el outer:  0.7685000000000001  +/-  0.05585696017507578  (95%)


El mejor modelo que me dio es el de SVC, ahora procedo a la busqueda de mejores hiperparametros

In [ ]:
svc= SVC(random_state=123)

# Creamos la grilla de hiperparametros a probar
param_grid_svc = [{'C':[0.0001,0.01,0.1,1], 'gamma':['scale','auto']}]

# Creamos el inner CV (GSCV para buscar los mejores hiperparámetros con los k folds internos)
gscv_svc = GridSearchCV(estimator=svc,
                    param_grid=param_grid_svc, 
                    scoring='accuracy',
                    n_jobs=-1,
                    cv=skfold_inner,
                    verbose=0,
                    refit=True)

In [32]:
algoritmo_elegido = gscv_svc
algoritmo_elegido.fit(x, y)

elegido_resultados = pd.DataFrame(algoritmo_elegido.cv_results_).loc[algoritmo_elegido.best_index_]

elegido_promedio = elegido_resultados.mean_test_score
elegido_desvio = elegido_resultados.std_test_score

print('AC: ', elegido_promedio*100, " +/- ", 2*elegido_desvio*100, " (95%)")